# Объекты культурного наследия народов Российской Федерации

## Введение

Используя [портал открытых данных](https://opendata.mkrf.ru/opendata/7705851331-egrkn) министерства культуры Российской Федерации, был получен датасет `data-51-structure-6.csv` со списком объектов культурного наследия.\
Это последняя, актуальная редакция, версия $51$.\
Предоставлены данные за декабрь $2023$ года.

**Цель:** провести предобработку датасета, чтобы он был пригодным для анализа.

**Задачи и план действий:**

- Обзор данных
- Предобработка данных
    - Работа с колонками
        - Удаление колонок с пропусками 90% и выше
        - Преобразование JSON данных
        - Удаление колонок, появившихся после преобразования JSON
- Обзор колонок
- Предобработка данных
    - Работа с колонками
        - Удаление ненужных колонок
        - Переименование колонок
        - Изменение типов данных
    - Пропущенные значения
    - Дублирующие значения
- Рефлексия
- Резюме по предобработке данных

<br>

## Обзор данных

Импортируем нужные библиотеки и зададим опции.

In [1]:
# импорт библиотек и модулей

# базовые
import pandas as pd
import numpy as np

# дополнительные
import json
import re

# подключение к google drive
from google.colab import drive
drive.mount('/content/drive')

# неограниченное количество колонок в выводе
pd.set_option('display.max_columns', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Прочитаем файл из облачного хранилища Google Drive.\
К сожалению, из-за очень большого размера csv-файла ($522$ Мб) возникают трудности с чтением файла при ином способе импорта.

In [2]:
# чтение файла из google drive
path = '/content/drive/MyDrive/Colab Datasets/data-51-structure-6.csv'
df = pd.read_csv(path, low_memory=False)

Посмотрим общий объём памяти в мегабайтах, который занимает датафрейм.

In [3]:
# метод находит общий объём в байтах, который занимает датафрейм;
# произведение в скобках высчитывает кол-во байт в одном мегабайте
mem_us_before = df.memory_usage(deep=True).sum() / (1024 * 1024)
mem_us_before

830.1200323104858

Результат: $830$ Мегабайтов.

Взглянем на таблицу.

In [4]:
df.head()

,Объект,Номер в реестре,название улицы,Комментарий,Идентификатор дома в ФИАС,FiasAuxSubobjId,FiasAuxObjId,Идентификатор улицы в ФИАС,Идентификатор населенного пункта в ФИАС,Идентификатор внутрегородской территории в ФИАС,Идентификатор города в ФИАС,Идентификатор городских и сельских поселений в ФИАС,Идентификатор района в ФИАС,Идентификатор автономного округа в ФИАС,Идентификатор региона в ФИАС,Идентификатор страны в ФИАС,Полный адрес,На карте,Id-Регион,Регион,учетный номер,Id-Категория историко-культурного значения,Категория историко-культурного значения,Id-Тип объекта культурного наследия,Тип объекта культурного наследия,Id-Вид объекта,Вид объекта,описание предмета охраны,текстовое описание границ,Id-общая видовая принадлежность,Id-Принадлежность к Юнеско,Принадлежность к Юнеско,Id-Особо ценный объект,Особо ценный объект,идентификатор ансамбля,дата создания,GeoId,X,Y,наименование документа,Изображение,зоны охраны и режимы использования земель,Items,Id - идентификатор,признак актуальности,признак публичности,год постановки ОКН на Госохрану,год проведения реставрации,"параметры, идентифицирующие ФЦП",относительный URL объекта,категория объекта,дата создания объекта,дата обновления объекта
0,"Дом, в котором в 1917 г. находился Совет рабоч...",671510201120006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Смоленская область. г. Смоленск, ул. Октябрьск...","{""coordinates"":[32.04383481349142,54.780875011...",67,Смоленская область,67-292,1,Федерального значения,NaN,NaN,1,Памятник,Предметом охраны объекта культурного наследия ...,NaN,"[{""id"":""2"",""value"":""Памятник истории""}]",2,нет,2,нет,NaN,NaN,NaN,NaN,NaN,"[{""name"":""Постановление Совета Министров РСФСР...","{""url"":""https://okn-mk.mkrf.ru/maps/show/id/12...",NaN,NaN,147449,NaN,NaN,NaN,NaN,NaN,/cdm/v2/heritages/1/147449,heritage,2015-10-02T06:46:44Z,NaN
1,Дом Ф.Д. Маштакова,571410013830005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Новосибирская область, г. Новосибирск, Централ...","{""coordinates"":[82.921885,55.022711],""type"":""P...",54,Новосибирская область,54-797,2,Регионального значения,NaN,NaN,1,Памятник,1. Объемно-планировочное решение и габариты зд...,Граница территории объекта культурного наследи...,"[{""id"":""3"",""value"":""Памятник градостроительств...",2,нет,2,нет,NaN,1903 г.,NaN,NaN,NaN,"[{""name"":""Постановление Главы Администрации Но...","{""url"":""https://okn-mk.mkrf.ru/maps/show/id/35...",Объект культурного наследия расположен в охран...,"[{""coordinates"":[[55.022675,82.92171667000001]...",147450,true,NaN,2001.0,NaN,NaN,/cdm/v2/heritages/1/147450,heritage,NaN,2022-01-31T10:47:19Z
2,Дом жилой,671510200540005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Смоленская область, г. Смоленск, ул. Коммунист...","{""coordinates"":[32.0563380846552,54.7802279058...",67,Смоленская область,67-939,2,Регионального значения,NaN,NaN,1,Памятник,Предметом охраны объекта культурного наследия ...,Граница территории идет: - от исходной поворо...,"[{""id"":""3"",""value"":""Памятник градостроительств...",2,нет,2,нет,NaN,нач. XX в.,NaN,NaN,NaN,"[{""name"":""Решение Смоленской областной Думы"",""...",NaN,NaN,NaN,147454,true,NaN,NaN,NaN,NaN,/cdm/v2/heritages/1/147454,heritage,NaN,2023-09-29T09:24:33Z
3,Памятник Александру Невскому,761310302740006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Ярославская область, г. Переславль-Залесский, ...",NaN,76,Ярославская область,76-554,1,Федерального значения,NaN,NaN,1,Памятник,Предметом охраны объекта культурного наследия ...,Граница территории объекта культурного наследи...,"[{""id"":""4"",""value"":""Памятник искусства""}]",2,нет,2,нет,NaN,1958 г.,NaN,NaN,NaN,"[{""name"":""Постановление Совета Министров РСФСР...","{""url"":""https://okn-mk.mkrf.ru/maps/show/id/41...",NaN,NaN,147455,true,NaN,NaN,NaN,NaN,/cdm/v2/heritages/1/147455,heritage,NaN,2021-12-23T17:14:05Z
4,"Здание городского торгового корпуса, где 14 де...",541510302250006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

Видно очень много пропусков. И ничего непонятно.

Известно, что одна строка означает один объект культурного наследия.

Посмотрим размер таблицы.

In [5]:
# сохраним в переменную, чтобы сравнить с конечным обработанным результатом
raw_shape = df.shape

# размер таблицы
print(f'Размер таблицы — {df.shape[0]} строк и {df.shape[1]} колонки.')

Размер таблицы — 154493 строк и 53 колонки.


Посмотрим названия колонок и их типы данных.

In [6]:
df.dtypes

Объект                                                  object
Номер в реестре                                         object
название улицы                                         float64
Комментарий                                            float64
Идентификатор дома в ФИАС                              float64
FiasAuxSubobjId                                        float64
FiasAuxObjId                                           float64
Идентификатор улицы в ФИАС                             float64
Идентификатор населенного пункта в ФИАС                float64
Идентификатор внутрегородской территории в ФИАС        float64
Идентификатор города в ФИАС                            float64
Идентификатор городских и сельских поселений в ФИАС    float64
Идентификатор района в ФИАС                            float64
Идентификатор автономного округа в ФИАС                float64
Идентификатор региона в ФИАС                           float64
Идентификатор страны в ФИАС                            

Полный хаос.\
Чтобы сделать адекватный обзор данных, нужно сначала провести их предобработку.

<br>

## Предобработка данных

### Работа с колонками

#### Удаление колонок с пропусками 90% и выше

Посмотрим, есть ли колонки, значения которых состоят полностью из пропусков.

In [7]:
len(df.columns[df.isna().all()])

19

19 таких колонок.

Их названия.

In [8]:
null_columns = df.columns[df.isna().all()]
null_columns

Index(['название улицы', 'Комментарий', 'Идентификатор дома в ФИАС',
       'FiasAuxSubobjId', 'FiasAuxObjId', 'Идентификатор улицы в ФИАС',
       'Идентификатор населенного пункта в ФИАС',
       'Идентификатор внутрегородской территории в ФИАС',
       'Идентификатор города в ФИАС',
       'Идентификатор городских и сельских поселений в ФИАС',
       'Идентификатор района в ФИАС',
       'Идентификатор автономного округа в ФИАС',
       'Идентификатор региона в ФИАС', 'Идентификатор страны в ФИАС',
       'Id-Тип объекта культурного наследия',
       'Тип объекта культурного наследия', 'GeoId', 'X', 'Y'],
      dtype='object')

Удалим колонки с пропусками во всех строках.

In [9]:
# удаление
df = df.drop(columns=null_columns)

Посмотрим, сколько колонок осталось в таблице.

In [10]:
print(f'Количество колонок: {len(df.columns)}')

Количество колонок: 34


<br>

Высветим колонки и процент пропусков в них.

In [11]:
null_percent = df.isna().mean() * 100
null_percent.sort_values(ascending=False)

признак публичности                           99.998705
параметры, идентифицирующие ФЦП               99.985113
год проведения реставрации                    99.858246
Items                                         94.979061
идентификатор ансамбля                        85.477659
зоны охраны и режимы использования земель     83.186293
год постановки ОКН на Госохрану               78.408730
На карте                                      74.141223
текстовое описание границ                     72.662192
описание предмета охраны                      69.882778
дата создания объекта                         62.023522
Изображение                                   39.187536
дата обновления объекта                       37.976478
Полный адрес                                  36.309736
Id-общая видовая принадлежность                2.292661
признак актуальности                           1.519810
дата создания                                  1.209116
наименование документа                         0

<br>

Снесём колонки, где процент пропусков $90$% и выше.

In [12]:
# удаление
df = df.drop(columns = null_percent[null_percent >= 90].index)

In [13]:
print(f'Теперь колонок стало {len(df.columns)}.')

Теперь колонок стало 30.


<br>

Взглянем ещё раз на данные одного объекта культурного наследия по вертикали.

In [14]:
# опция для просмотра всего содержимого колонок
pd.set_option('display.max_colwidth', None)

display(df[df['Номер в реестре'] == '251410132170005'].transpose())

# вернуть ширину колонок по умолчанию
pd.reset_option('display.max_colwidth')

,13505
Объект,Дом казака Евстафия Коренева
Номер в реестре,251410132170005
Полный адрес,"Приморский край, Пограничный район, поселок Пограничный, Советская ул., 25"
На карте,"{""coordinates"":[131.37387504662703,44.4118772191358],""type"":""Point""}"
Id-Регион,25
Регион,Приморский край
учетный номер,25-38194
Id-Категория историко-культурного значения,2
Категория историко-культурного значения,Регионального значения
Id-Вид объекта,1


<br>

Если обратиться к порталу открытых данных министерства культуры РФ и посмотреть этот объект на сайте, станет ясно, что в этом датафрейме некоторые колонки слиплись в одну.

Мы столкнулись с JSON-данными, которые содержат вложенные структуры. Мы видим словари с ключами и их значениями, где ключи — это название колонки, а значение — это значение колонки.

Из среза по номеру в реестре выше известно, что колонка `наименование документа` содержит в себе значения пяти колонок сразу.\
Это заметно по английским словам, ограниченным двойными кавычками, после которых стоит двоеточие.\
Так, в этой колонке видим $5$ отдельных колонок: `name`, `number`, `date`, `archive_id`, `archive_url`.\
Это выражено сложной структурой: список, внутри которого словарь, а значением одного из ключей словаря является ещё один словарь.

Аналогичную проблему видим в колонке `Id-общая видовая принадлежность`.\
Разделители подсказывают нам, что эту колонку нужно поделить на 2 отдельные: `id` и `value`.

Такая же проблема в колонке `Изображение`.\
Там деление на 3 колонки: `url`, `preview` и `title`.

С колонкой `На карте` всё тоже самое.\
Она делится на 2 колонки: `coordinates` и `type`, причём ключи `coordinates` содержат списки.

Всё это нужно исправить должным образом.

Конечно же мне будет помогать в этом ChatGPT.\
Потому что я не гений, а всего лишь человек, которому нравится то, что он сейчас делает.

<br>

#### Преобразование JSON-данных

##### Преобразование колонки `На карте`

Сначала напишем функцию.

In [15]:
# Функция для разбора значений в колонке 'На карте'
def parse_coordinates(row):
    try:
        # Шаг 1: Проверка, не является ли `row` NaN
        if pd.notnull(row):
            # Шаг 2: Преобразование строки в формате JSON в объект Python
            json_data = json.loads(row)
            # Шаг 3: Возвращение объекта типа pd.Series с данными из JSON
            return pd.Series(json_data, dtype='object')
    # Шаг 4: Обработка определённых ошибок, если они сработают
    except (json.JSONDecodeError, TypeError):
        # пропуск, если ошибки сработали
        pass
    # Шаг 5: Возвращение пустого объекта pd.Series с dtype='object', если `row` оказался Nan или сработали ошибки
    return pd.Series(dtype='object')

Затем применим её. Получившиеся новые колонки объединим с датафреймом.

In [16]:
# применение функции parse_coordinates к каждому элементу колонки 'На карте' и создание новых колонок
parsed_coordinates = df['На карте'].apply(parse_coordinates)

# объединение датафрейма с новыми колонками, по горизонтали вдоль колонок
df = pd.concat([df, parsed_coordinates], axis=1)

# удаление исходной колонки
df.drop('На карте', axis=1, inplace=True)

<br>

##### Преобразование колонки `Id-общая видовая принадлежность`

In [17]:
# преобразование JSON-строки в столбце "Id-общая видовая принадлежность" в две новые колонки

# x - это JSON-строка, которая представляет собой список словарей
# 0 - это первый элемент списка в виде словаря, который содержит ключи "id" и "value"

# применяем функцию lambda к колонке "Id-общая видовая принадлежность":
# - если значение не является NaN, то используем json.loads() для преобразования JSON-строки в список словарей,
#   затем извлекаем значение ключа 'id' из первого элемента в списке и присваиваем его новой колонке
# - если значение является NaN, присваиваем None новой колонке
df['id-общая видовая принадлежность'] = df['Id-общая видовая принадлежность'].apply(lambda x: json.loads(x)[0]['id'] if pd.notnull(x) else None)

# применяем аналогичную функцию lambda
# извлекаем значение ключа 'value' из первого элемента в списке и присваиваем новой колонке
df['общая видовая принадлежность'] = df['Id-общая видовая принадлежность'].apply(lambda x: json.loads(x)[0]['value'] if pd.notnull(x) else None)

# удаление исходной колонки
df.drop('Id-общая видовая принадлежность', axis=1, inplace=True)

<br>

##### Преобразование колонки `наименование документа`

Сначала напишем функцию.

In [18]:
# Функция для разбора значений в колонке 'наименование документа'
def extract_info(row):
    try:
        # Шаг 1: Проверка, не является ли `row` NaN
        if pd.notnull(row):
            # Шаг 2: Преобразование строки в формате JSON в объект Python
            json_data = json.loads(row)
            # Шаг 3: Проверка, являются ли данные списком и имеет ли он хотя бы один элемент
            # ф-я isinstance в Python используется для проверки принадлежности объекта (json_data) к определенному типу данных (list)
            if isinstance(json_data, list) and len(json_data) > 0:
                # пустой словарь result, для хранения преобразованных данных
                result = {}
                # Шаг 4: Перебор элементов списка, элементы которого — словари
                # enumerate перебирает элементы списка json_data, возвращая кортежи (индекс, элемент) на каждой итерации, где:
                # i - переменная, которая будет содержать текущий индекс элемента в списке,
                # item - переменная, которая будет содержать текущий элемент списка
                for i, item in enumerate(json_data):
                    # Шаг 5: Перебор пар ключ-значение в словаре методом для словарей items()
                    for key, value in item.items():
                        # Шаг 6: Формирование нового ключа для каждого элемента
                        # путем объединения оригинального ключа key и индекса i + 1.
                        # индекс i + 1 используется, чтобы ключи были уникальными для каждого элемента списка
                        new_key = f"{key}_{i + 1}"
                        # Шаг 7: Если значение словарь, то разбор вложенных значений:
                        if isinstance(value, dict):
                            # перебирает его пары ключ-значение с помощью value.items()
                            # для каждой пары создается новый ключ sub_new_key, который объединяет исходный ключ new_key и ключ из вложенного словаря sub_key.
                            # этот новый ключ используется для сохранения соответствующего значения в результирующем словаре result.
                            for sub_key, sub_value in value.items():
                                sub_new_key = f"{new_key}_{sub_key}"
                                result[sub_new_key] = sub_value
                        # если значение не является словарем (то есть оно не имеет вложенных данных), оно сохраняется в результирующем словаре result с ключом new_key.
                        else:
                            result[new_key] = value
                # Шаг 8: возврат результатов в виде объекта Series
                # явно указываем dtype='object', поскольку JSON может содержать данные различных типов (строки, числа  и т. д.)
                return pd.Series(result, dtype='object')
    # Шаг 9: Обработка исключений, если произошла ошибка при разборе JSON строки или типа данных
    except (json.JSONDecodeError, TypeError):
    # пропуск, если сработали определенные ошибки
        pass
    # Шаг 10: Возвращение пустого объекта pd.Series с dtype='object', если `row` оказался Nan или сработали ошибки
    return pd.Series(dtype='object')

Затем применим её. Получившиеся новые колонки объединим с датафреймом.

In [19]:
# применение функции extract_info к каждому элементу колонки 'наименование документа' и создание новых колонок
doc_info = df['наименование документа'].apply(extract_info)

# объединение doc_info с основным DataFrame
df = pd.concat([df, doc_info], axis=1)

# удаление исходной колонки
df.drop('наименование документа', axis=1, inplace=True)

<br>

##### Преобразование колонки `Изображение`

Сначала напишем функцию.

In [20]:
# преобразование колонки в str тип
df['Изображение'] = df['Изображение'].astype(str)

# Функция для разбора значений в колонке 'Изображение'
def parse_images(row):
    try:
        # преобразование строки формата JSON в объект Python
        return json.loads(row)
    # обработка ошибки, если она сработает:
    except json.JSONDecodeError:
        # вернуть пустой словарь
        return {}

Затем применим её и получим новые колонки.

In [21]:
# преобразование значений в колонке "Изображение" из JSON в словари
df['Изображение'] = df['Изображение'].apply(parse_images)

# создание двух новых колонок: 'URL' и 'Title' путём извлечения соответствующих значений из словарей в колонке 'Изображение'
# используется метод get() для безопасного извлечения значений, чтобы избежать ошибок, если ключи 'url' или 'title' отсутствуют в словаре
df['URL'] = df['Изображение'].apply(lambda x: x.get('url', ''))
df['Title'] = df['Изображение'].apply(lambda x: x.get('title', ''))

# удаление исходной колонки
df.drop('Изображение', axis=1, inplace=True)

Наконец, спустя где-то часов $20$ совместного разбора с ChatGPT, колонки преобразованы.\
Настоящее безумие.

<br>

Сделаем срез по городу Ухта.

In [22]:
df[df['Полный адрес'].str\
                     .contains('Ухта', na = False)]\
                     .sort_values(by='Объект')\
                     .head()

,Объект,Номер в реестре,Полный адрес,Id-Регион,Регион,учетный номер,Id-Категория историко-культурного значения,Категория историко-культурного значения,Id-Вид объекта,Вид объекта,описание предмета охраны,текстовое описание границ,Id-Принадлежность к Юнеско,Принадлежность к Юнеско,Id-Особо ценный объект,Особо ценный объект,идентификатор ансамбля,дата создания,зоны охраны и режимы использования земель,Id - идентификатор,признак актуальности,год постановки ОКН на Госохрану,относительный URL объекта,категория объекта,дата создания объекта,дата обновления объекта,coordinates,type,id-общая видовая принадлежность,общая видовая принадлежность,name_1,number_1,date_1,archive_1_id,archive_1_url,name_2,number_2,date_2,archive_2_id,archive_2_url,name_3,number_3,date_3,archive_3_id,archive_3_url,name_4,number_4,date_4,archive_4_id,archive_4_url,name_5,number_5,date_5,archive_5_id,archive_5_url,name_6,number_6,date_6,archive_6_id,archive_6_url,name_7,number_7,date_7,archive_7_id,archive_7_url,name_8,number_8,date_8,archive_8_id,archive_8_url,name_9,number_9,date_9,archive_9_id,archive_9_url,name_10,number_10,date_10,archive_10_id,archive_10_url,name_11,number_11,date_11,archive_11_id,archive_11_url,URL,Title
147528,"Ансамбль улиц Мира, Первомайская, Кремса, архи...",112021333140005,"Республика Коми, г. Ухта, ул. Мира, дома №№ 1,...",11,Республика Коми,11-168149,2,Регионального значения,2,Ансамбль,NaN,Граница территории объекта культурного наследи...,2,нет,2,нет,NaN,1949 - 1968 гг.,NaN,324258,true,NaN,/cdm/v2/heritages/2/324258,heritage,NaN,2023-07-20T07:34:53Z,NaN,NaN,3,Памятник градостроительства и архитектуры,Приказ Управления Республики Коми по охране об...,93-ОД,2019-06-17,NaN,NaN,Приказ Управления Республики Коми по охране об...,116-ОД,2020-07-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/4034367,4034367
147483,"Ансамбль улиц Октябрьская и Первомайская, архи...",112021333120005,"Республика Коми, г. Ухта, ул. Октябрьская, дом...",11,Республика Коми,11-171869,2,Регионального значения,2,Ансамбль,Предметом охраны объекта культурного наследия...,NaN,2,нет,2,нет,NaN,1950 - 1959 гг.,NaN,324108,true,NaN,/cdm/v2/heritages/2/324108,heritage,NaN,2023-07-20T07:41:01Z,NaN,NaN,3,Памятник градостроительства и архитектуры,Приказ Управления Республики Коми по охране об...,167-ОД,2019-08-05,NaN,NaN,Приказ Управления Республики Коми по охране об...,142-ОД,2020-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/4028963,4028963
149208,Больница поселка Водный,112111343810005,"Республика Коми, г. Ухта, пгт. Водный, ул. Гаг...",11,Республика Коми,11-179117,2,Регионального значения,1,Памятник,1 Объемно-пространственная композиция здания ...,Граница территории объекта культурного наследи...,2,нет,2,нет,NaN,1956 г.,NaN,328787,true,2021.0,/cdm/v2/heritages/1/328787,heritage,2022-07-05T14:33:03Z,NaN,NaN,NaN,3,Памятник градостроительства и архитектуры,Приказ Управления Республики коми по охране об...,20-ОД,2021-03-05,NaN,NaN,Приказ Управления Республики Коми по охране об...,36-ОД,2021-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/4396147,4396147
71930,Братская могила 12 красных партизан в с. Изваиле,111610537090005,"Республика Коми, г. Ухта, д. Изваиль (ориентир...",11,Республика Коми,11-82351,2,Регионального значения,1,Памятник,"К особенностям, составляющим предмет охраны об...",NaN,2,нет,2,нет,NaN,"ноябрь 1919 г., ноябрь 1957 г.",NaN,233259,true,1959.0,/cdm/v2/heritages/1/233259,heritage,NaN,2021-12-23T18:39:51Z,NaN,NaN,2,Памятник 

Колонки `Изображение`, `наименование документа`, `Id-общая видовая принадлежность` и `На карте` исчезли, а вместо них появились другие.

Если вглядеться, то образовалось много колонок-паразитов, в названиях которых присутствуют числовые суффиксы от 1 до 11.\
Если обратиться к сайту минкульта, то станет ясно, что это поправки в документах.

<br>

#### Удаление колонок-паразитов

Посмотрим, каков процент пропусков в этих новых колонках.

In [23]:
null_percent = df.isna().mean() * 100
null_percent.sort_values(ascending=False).head(20)

number_9          99.958574
archive_10_url    99.958574
number_10         99.958574
name_10           99.958574
archive_9_url     99.958574
date_9            99.958574
date_10           99.958574
archive_10_id     99.958574
archive_9_id      99.958574
name_11           99.958574
number_11         99.958574
date_11           99.958574
archive_11_id     99.958574
archive_11_url    99.958574
name_9            99.958574
date_8            99.957927
number_8          99.957927
name_8            99.957927
archive_8_url     99.957927
archive_8_id      99.957927
dtype: float64

In [24]:
print(f'Количество колонок: {df.shape[1]}')

Количество колонок: 87


<br>

Видим много колонок с $99$% пропусков. Нужно их снести.

In [25]:
# удаление
df = df.drop(columns = null_percent[null_percent >= 90].index)

<br>

Проверим, уменьшилось ли их количество.

In [26]:
print(f'Количество колонок: {df.shape[1]}')

Количество колонок: 42


Колонки удалены.

<br>

## Обзор колонок

Разберёмся, что вообще значит каждая колонка в датафрейме.

Выведем названия всех колонок.

In [27]:
df.columns.to_list()

['Объект',
 'Номер в реестре',
 'Полный адрес',
 'Id-Регион',
 'Регион',
 'учетный номер',
 'Id-Категория историко-культурного значения',
 'Категория историко-культурного значения',
 'Id-Вид объекта',
 'Вид объекта',
 'описание предмета охраны',
 'текстовое описание границ',
 'Id-Принадлежность к Юнеско',
 'Принадлежность к Юнеско',
 'Id-Особо ценный объект',
 'Особо ценный объект',
 'идентификатор ансамбля',
 'дата создания',
 'зоны охраны и режимы использования земель',
 'Id - идентификатор',
 'признак актуальности',
 'год постановки ОКН на Госохрану',
 'относительный URL объекта',
 'категория объекта',
 'дата создания объекта',
 'дата обновления объекта',
 'coordinates',
 'type',
 'id-общая видовая принадлежность',
 'общая видовая принадлежность',
 'name_1',
 'number_1',
 'date_1',
 'archive_1_id',
 'archive_1_url',
 'name_2',
 'number_2',
 'date_2',
 'archive_2_id',
 'archive_2_url',
 'URL',
 'Title']

Имея под рукой отдельный объект культурного наследия, открытый на сайте минкульта РФ, получили информацию.\
Одна строка в датафрейме — это один объект культурного наследия. Эта строка содержит сведения об объекте.\
Чтобы описать колонку более точно, будем использовать метод `value_counts`.

<br>

Итак,\
`Объект` — это сам объект культурного наследия Российской федерации, его наименование.

In [28]:
df['Объект'].value_counts(dropna=False).head()

Дом жилой           4021
Жилой дом           2113
Курган              1381
Курганная группа     887
Селище               755
Name: Объект, dtype: int64

<br>

`Номер в реестре` — это номер объекта, по которому его можно идентифицировать на сайте открытых данных минкульта. У каждого объекта свой уникальный номер в реестре.

In [29]:
df['Номер в реестре'].value_counts(dropna=False).head()

671510201120006    1
131710904690005    1
131710904710005    1
581710920830005    1
131710904720005    1
Name: Номер в реестре, dtype: int64

<br>

`Id-Регион` — это код субъекта РФ.

In [30]:
df['Id-Регион'].value_counts(dropna=False).head()

61    9658
23    9264
69    7398
5     6369
77    6228
Name: Id-Регион, dtype: int64

<br>

`Регион` — это субъект РФ.

In [31]:
df['Регион'].value_counts(dropna=False).head()

Ростовская область     9658
Краснодарский край     9264
Тверская область       7398
Республика Дагестан    6369
г. Москва              6228
Name: Регион, dtype: int64

<br>

`учетный номер` — некий учётный номер. Возможно, служебная информация. У каждого объекта свой учётный номер.

In [32]:
df['учетный номер'].value_counts(dropna=False).head()

67-292       1
13-77250     1
13-77258     1
58-124112    1
13-77260     1
Name: учетный номер, dtype: int64

<br>

`Id-Категория историко-культурного значения` — идентификатор категории историко-культурного значения. Идентификаторов всего три.

In [33]:
df['Id-Категория историко-культурного значения'].value_counts(dropna=False).head()

2    78696
1    71746
4     4051
Name: Id-Категория историко-культурного значения, dtype: int64

<br>

`Категория историко-культурного значения` — название категории. Их также три:
* объект муниципального значения
* объект регионального значения
* объект федерального значения

In [34]:
df['Категория историко-культурного значения'].value_counts(dropna=False).head()

Регионального значения                78696
Федерального значения                 71746
Местного (муниципального) значения     4051
Name: Категория историко-культурного значения, dtype: int64

<br>

`Id-Вид объекта` —  идентификатор вида объекта. Их всего три.

In [35]:
df['Id-Вид объекта'].value_counts(dropna=False).head()

1    129762
2     22382
3      2349
Name: Id-Вид объекта, dtype: int64

<br>

`Вид объекта` — название вида объекта. Их 3:
* Памятник
* Ансамбль
* Достопримечательное место

In [36]:
df['Вид объекта'].value_counts(dropna=False).head()

Памятник                     129762
Ансамбль                      22382
Достопримечательное место      2349
Name: Вид объекта, dtype: int64

<br>

`описание предмета охраны` — описание объекта культурного наследия. Видно очень много пропусков и много неявных дубликатов.

In [37]:
df['описание предмета охраны'].value_counts(dropna=False).head()

NaN                             107964
не утвержден                      1272
Не утвержден.                     1212
Не утвержден                       792
Предмет охраны не утвержден.       658
Name: описание предмета охраны, dtype: int64

<br>

`текстовое описание границ` — обозначение границ объекта текстом. Очень много пропусков и много неявных дубликатов.

In [38]:
df['текстовое описание границ'].value_counts(dropna=False).head()

NaN                                  112258
Не утверждены.                         1203
не утверждены                           588
Границы территории не утверждены        452
Границы территории не утверждены.       432
Name: текстовое описание границ, dtype: int64

<br>

`Id-Принадлежность к Юнеско` — идентификатор, принадлежит ли объект к объектам всемирного наследия Юнеско. Вероятно, представляет собой корявое булево значение.

In [39]:
df['Id-Принадлежность к Юнеско'].value_counts(dropna=False).head()

2    153534
1       948
0        11
Name: Id-Принадлежность к Юнеско, dtype: int64

<br>

`Принадлежность к Юнеско` — текстовое указание принадлежности к Юнеско.

In [40]:
df['Принадлежность к Юнеско'].value_counts(dropna=False).head()

нет    153534
да        948
NaN        11
Name: Принадлежность к Юнеско, dtype: int64

<br>

`Id-Особо ценный объект` — идентификатор, является ли объект особо ценным.

In [41]:
df['Id-Особо ценный объект'].value_counts(dropna=False).head()

2    154217
1       265
0        11
Name: Id-Особо ценный объект, dtype: int64

<br>

`Особо ценный объект` — является ли объект особо ценным.

In [42]:
df['Особо ценный объект'].value_counts(dropna=False).head()

нет    154217
да        265
NaN        11
Name: Особо ценный объект, dtype: int64

<br>

`идентификатор ансамбля` — идентификатор принадлежности объекта к определённому ансамблю. Очень много пропусков.

In [43]:
df['идентификатор ансамбля'].value_counts(dropna=False).head()

NaN         132057
151344.0       219
164457.0       147
214508.0       121
194299.0       107
Name: идентификатор ансамбля, dtype: int64

<br>

`дата создания` — дата создания объекта культурного наследия.

In [44]:
df['дата создания'].value_counts(dropna=False).head()

дата создания (возникновения) не определена    8114
XIX в.                                         4027
III тыс. до н.э. - XIV в. н.э.                 3096
Дата создания (возникновения) не определена    2236
1941-1945 гг.                                  2149
Name: дата создания, dtype: int64

<br>

`зоны охраны и режимы использования земель` — определение зоны охраны объекта. Очень много пропусков и много неявных дублей.

In [45]:
df['зоны охраны и режимы использования земель'].value_counts(dropna=False).head()

NaN                        128517
Не утверждены.               2488
не утверждены                1316
Не утверждены                 744
зоны охраны установлены       655
Name: зоны охраны и режимы использования земель, dtype: int64

<br>

`Id - идентификатор` — уникальный ID объекта.

In [46]:
df['Id - идентификатор'].value_counts(dropna=False).head()

147449    1
264705    1
264707    1
264708    1
264709    1
Name: Id - идентификатор, dtype: int64

<br>

`признак актуальности` — что-то непонятное и странное. Актуален ли объект или нет?

In [47]:
df['признак актуальности'].value_counts(dropna=False).head()

true    152143
NaN       2348
on           2
Name: признак актуальности, dtype: int64

<br>

`год постановки ОКН на Госохрану` — поле говорит само за себя. Очень много пропусков.

In [48]:
df['год постановки ОКН на Госохрану'].value_counts(dropna=False).head()

NaN       121136
1997.0      2738
1993.0      2314
1974.0      2301
2000.0      1670
Name: год постановки ОКН на Госохрану, dtype: int64

<br>

`относительный URL объекта` — относительный адрес объекта на сервере открытых данных. У каждого объекта свой URL.

In [49]:
df['относительный URL объекта'].value_counts(dropna=False).head()

/cdm/v2/heritages/1/147449    1
/cdm/v2/heritages/1/264705    1
/cdm/v2/heritages/1/264707    1
/cdm/v2/heritages/1/264708    1
/cdm/v2/heritages/1/264709    1
Name: относительный URL объекта, dtype: int64

<br>

`категория объекта` — видимо, служебная информация. Одно и то же значение повторяется всё время.

In [50]:
df['категория объекта'].value_counts(dropna=False).head()

heritage    154493
Name: категория объекта, dtype: int64

<br>

`дата создания объекта` — что-то непонятное. Возможно, это дата внесения в реестр объекта культурного наследия. Диапазон этих дат от 2015 до конца 2023 года. Очень много пропусков.

In [51]:
df['дата создания объекта'].value_counts(dropna=False).head()

NaN                     95822
2018-02-26T16:04:39Z        3
2023-10-11T11:58:51Z        2
2018-09-11T08:14:26Z        2
2023-11-10T12:57:42Z        2
Name: дата создания объекта, dtype: int64

In [52]:
df['дата создания объекта'].sort_values().head()

257     2015-05-07T16:10:26Z
2042    2015-05-19T12:29:24Z
2364    2015-05-19T12:36:08Z
652     2015-05-19T12:36:10Z
281     2015-05-19T12:36:27Z
Name: дата создания объекта, dtype: object

In [53]:
df['дата создания объекта'].sort_values(ascending=False).head()

154492    2023-12-19T14:52:08Z
154491    2023-12-19T14:52:05Z
154490    2023-12-19T14:52:02Z
154489    2023-12-19T14:51:59Z
154488    2023-12-19T14:51:56Z
Name: дата создания объекта, dtype: object

<br>

`дата обновления объекта` — возможно, это дата обновления объекта в реестре. Диапазон дат от 2018 до конца 2023. Очень много пропусков.

In [54]:
df['дата обновления объекта'].value_counts(dropna=False).head()

NaN                     58671
2023-10-09T14:55:20Z        4
2023-10-12T14:22:53Z        4
2021-12-23T18:53:34Z        3
2023-12-01T11:46:53Z        3
Name: дата обновления объекта, dtype: int64

In [55]:
df['дата обновления объекта'].sort_values().head()

37061     2018-06-26T09:02:46Z
132737    2018-06-26T09:10:35Z
13333     2018-06-26T13:13:20Z
11439     2018-06-26T13:14:13Z
13320     2018-06-26T13:14:22Z
Name: дата обновления объекта, dtype: object

In [56]:
df['дата обновления объекта'].sort_values(ascending=False).head()

148521    2023-12-21T11:50:32Z
29029     2023-12-21T11:50:31Z
28924     2023-12-21T11:50:30Z
28849     2023-12-21T11:50:29Z
28833     2023-12-21T11:50:28Z
Name: дата обновления объекта, dtype: object

<br>

`coordinates` — координаты объекта (ширина, долгота), отмеченные в реестре. Очень много пропусков.

In [57]:
df['coordinates'].astype(str)\
                 .value_counts(dropna=False)\
                 .head()

nan                                     114543
[45.450136, 42.707692]                      69
[49.1068469365817, 55.7999436090671]        47
[34.9578965608516, 44.8417174338509]        45
[37.788144, 55.693409]                      39
Name: coordinates, dtype: int64

<br>

`type` — отметка объекта на карте. Очень много пропусков — столько же, сколько пропущенных координат.

In [58]:
df['type'].value_counts(dropna=False).head()

NaN      114543
Point     39950
Name: type, dtype: int64

<br>

`id-общая видовая принадлежность` — идентификатор видовой принадлежности.

In [59]:
df['id-общая видовая принадлежность'].value_counts(dropna=False).head()

3       56310
1       56096
2       35002
None     3542
4        3442
Name: id-общая видовая принадлежность, dtype: int64

<br>

`общая видовая принадлежность` — видовая принадлежность, к которой относится объект культурного наследия:
* Памятник градостроительства и архитектуры
* Памятник археологии
* Памятник истории
* Памятник искусства

In [60]:
df['общая видовая принадлежность'].value_counts(dropna=False).head()

Памятник градостроительства и архитектуры    56310
Памятник археологии                          56096
Памятник истории                             35002
None                                          3542
Памятник искусства                            3442
Name: общая видовая принадлежность, dtype: int64

<br>

`name_1` — наименования документов, которые являются основанием для возведения объекта в объект культурного наследия.

In [61]:
df['name_1'].value_counts(dropna=False).head()

Закон Краснодарского края «О перечне объектов культурного наследия (памятников истории и культуры), расположенных на территории Краснодарского края»                                           8182
Постановление                                                                                                                                                                                  3503
Постановление Главы Администрации Ростовской области «О принятии на государственную охрану памятников истории и культуры области и мерах по их охране»                                         3485
Постановление Законодательного собрания Тверской области                                                                                                                                       3415
Постановление Псковского областного Собрания депутатов «Об утверждении государственного списка недвижимых памятников истории и культуры, подлежащих охране как памятники местного значения»    3010
Name: name_1, dtype:

<br>

`number_1` — номера документов.

In [62]:
df['number_1'].value_counts(dropna=False).head()

313-КЗ    8539
1327      5995
647       4901
51        4679
624       3744
Name: number_1, dtype: int64

<br>

`date_1` — дата создания документа.

In [63]:
df['date_1'].value_counts(dropna=False).head()

2000-08-17    8542
1960-08-30    6006
1997-11-27    4883
1997-02-21    4556
1974-12-04    3748
Name: date_1, dtype: int64

<br>

`archive_1_id` — по-видимому, это идентификатор документа, являющегося основанием для установки объекта культурного наследия.

In [64]:
df['archive_1_id'].value_counts(dropna=False)

NaN          58478
748077.0       371
459615.0       299
2137890.0      283
2113631.0      271
             ...  
473320.0         1
1370748.0        1
471288.0         1
470990.0         1
5199868.0        1
Name: archive_1_id, Length: 89641, dtype: int64

<br>

`archive_1_url` — это ссылка на документ, являющегося основанием для установки объекта культурного наследия.

In [65]:
df['archive_1_url'].value_counts(dropna=False)

NaN                                            58475
None                                           56096
https://okn-mk.mkrf.ru/maps/show/id/2137890      283
https://okn-mk.mkrf.ru/maps/show/id/2113631      269
https://okn-mk.mkrf.ru/maps/show/id/748077       164
                                               ...  
https://okn-mk.mkrf.ru/maps/show/id/3725370        1
https://okn-mk.mkrf.ru/maps/show/id/3725298        1
https://okn-mk.mkrf.ru/maps/show/id/3725455        1
https://okn-mk.mkrf.ru/maps/show/id/3722925        1
https://okn-mk.mkrf.ru/maps/show/id/5131480        1
Name: archive_1_url, Length: 37495, dtype: int64

<br>

Колонки `name_2`, `number_2`, `date_2`, `archive_2_id`, `archive_2_url` носят ту же функцию, что и предыдущие колонки с суффиксами-цифрой $1$.\
Это документы, вносящие правки.

<br>

`URL` — ссылка на изображение объекта культурного наследия. Первое значение встречается свыше шестидесяти тысяч раз. Это завуалированный пропуск.

In [66]:
df['URL'].value_counts(dropna=False)

                                               60542
https://okn-mk.mkrf.ru/maps/show/id/1836648        6
https://okn-mk.mkrf.ru/maps/show/id/1780885        3
https://okn-mk.mkrf.ru/maps/show/id/1793099        3
https://okn-mk.mkrf.ru/maps/show/id/4546450        2
                                               ...  
https://okn-mk.mkrf.ru/maps/show/id/905079         1
https://okn-mk.mkrf.ru/maps/show/id/904752         1
https://okn-mk.mkrf.ru/maps/show/id/904479         1
https://okn-mk.mkrf.ru/maps/show/id/904675         1
https://okn-mk.mkrf.ru/maps/show/id/5215584        1
Name: URL, Length: 93935, dtype: int64

<br>

`Title` — символическое название изображения в виде чисел. Первое значение также показно в виде скрытого пропуска.

In [67]:
df['Title'].value_counts(dropna=False)

           60542
1836648        6
1780885        3
1793099        3
4546450        2
           ...  
905079         1
904752         1
904479         1
904675         1
5215584        1
Name: Title, Length: 93935, dtype: int64

<br>

Вот мы и узнали, что внутри колонок.

Нужно принять решение, какие из них нужно оставить, а какие убрать.

<br>

## Предобработка данных

### Работа с колонками

#### Удаление ненужных колонок

Итак, подумав, принял решение, что следующие колонки нужно убрать, эта информация шумна и точно не понадобится в рамках проекта.

In [68]:
# шумные колонки
garbage_columns = df[['Id-Регион',
                      'учетный номер',
                      'Id-Категория историко-культурного значения',
                      'Id-Вид объекта',
                      'Id-Принадлежность к Юнеско',
                      'Id-Особо ценный объект',
                      'зоны охраны и режимы использования земель',
                      'Id - идентификатор',
                      'признак актуальности',
                      'год постановки ОКН на Госохрану',
                      'относительный URL объекта',
                      'категория объекта',
                      'дата создания объекта',
                      'дата обновления объекта',
                      'type',
                      'id-общая видовая принадлежность',
                      'archive_1_id',
                      'name_2',
                      'number_2',
                      'date_2',
                      'archive_2_id',
                      'archive_2_url',
                      'Title']].columns
garbage_columns

Index(['Id-Регион', 'учетный номер',
       'Id-Категория историко-культурного значения', 'Id-Вид объекта',
       'Id-Принадлежность к Юнеско', 'Id-Особо ценный объект',
       'зоны охраны и режимы использования земель', 'Id - идентификатор',
       'признак актуальности', 'год постановки ОКН на Госохрану',
       'относительный URL объекта', 'категория объекта',
       'дата создания объекта', 'дата обновления объекта', 'type',
       'id-общая видовая принадлежность', 'archive_1_id', 'name_2', 'number_2',
       'date_2', 'archive_2_id', 'archive_2_url', 'Title'],
      dtype='object')

<br>

Избавимся от них.

In [69]:
df = df.drop(columns = garbage_columns)

<br>

#### Переименование колонок

Теперь нужно упорядочить колонки и переименовать их, чтобы соответствовать хорошему стилю.

Вот их список.

In [70]:
df.columns

Index(['Объект', 'Номер в реестре', 'Полный адрес', 'Регион',
       'Категория историко-культурного значения', 'Вид объекта',
       'описание предмета охраны', 'текстовое описание границ',
       'Принадлежность к Юнеско', 'Особо ценный объект',
       'идентификатор ансамбля', 'дата создания', 'coordinates',
       'общая видовая принадлежность', 'name_1', 'number_1', 'date_1',
       'archive_1_url', 'URL'],
      dtype='object')

<br>

Переименуем их и выведем датафрейм.

In [71]:
df = df.rename(columns={'Объект': 'object',
                        'Номер в реестре': 'registry_number',
                        'Полный адрес': 'full_address',
                        'Регион': 'region',
                        'Категория историко-культурного значения': 'significance_category',
                        'Вид объекта': 'object_type',
                        'описание предмета охраны': 'object_description',
                        'текстовое описание границ': 'object_border_description',
                        'Принадлежность к Юнеско': 'unesco_affiliation',
                        'Особо ценный объект': 'particularly_valuable_object',
                        'идентификатор ансамбля': 'ensemble_id',
                        'дата создания': 'construction_date',
                        'общая видовая принадлежность': 'object_subtype',
                        'name_1': 'document_name',
                        'number_1': 'document_code',
                        'date_1': 'document_date',
                        'archive_1_url': 'document_url',
                        'URL': 'image_url'})
df.head()

,object,registry_number,full_address,region,significance_category,object_type,object_description,object_border_description,unesco_affiliation,particularly_valuable_object,ensemble_id,construction_date,coordinates,object_subtype,document_name,document_code,document_date,document_url,image_url
0,"Дом, в котором в 1917 г. находился Совет рабоч...",671510201120006,"Смоленская область. г. Смоленск, ул. Октябрьск...",Смоленская область,Федерального значения,Памятник,Предметом охраны объекта культурного наследия ...,NaN,нет,нет,NaN,NaN,"[32.04383481349142, 54.78087501154608]",Памятник истории,Постановление Совета Министров РСФСР,624,1974-12-04,https://okn-mk.mkrf.ru/maps/show/id/122158,https://okn-mk.mkrf.ru/maps/show/id/122155
1,Дом Ф.Д. Маштакова,571410013830005,"Новосибирская область, г. Новосибирск, Централ...",Новосибирская область,Регионального значения,Памятник,1. Объемно-планировочное решение и габариты зд...,Граница территории объекта культурного наследи...,нет,нет,NaN,1903 г.,"[82.921885, 55.022711]",Памятник градостроительства и архитектуры,Постановление Главы Администрации Новосибирско...,1303,2001-12-29,https://okn-mk.mkrf.ru/maps/show/id/122120,https://okn-mk.mkrf.ru/maps/show/id/3599850
2,Дом жилой,671510200540005,"Смоленская область, г. Смоленск, ул. Коммунист...",Смоленская область,Регионального значения,Памятник,Предметом охраны объекта культурного наследия ...,Граница территории идет: - от исходной поворо...,нет,нет,NaN,нач. XX в.,"[32.0563380846552, 54.7802279058219]",Памятник градостроительства и архитектуры,Решение Смоленской областной Думы,24,1995-02-28,https://okn-mk.mkrf.ru/maps/show/id/122245,
3,Памятник Александру Невскому,761310302740006,"Ярославская область, г. Переславль-Залесский, ...",Ярославская область,Федерального значения,Памятник,Предметом охраны объекта культурного наследия ...,Граница территории объекта культурного наследи...,нет,нет,NaN,1958 г.,NaN,Памятник искусства,Постановление Совета Министров РСФСР,1327,1980-08-30,NaN,https://okn-mk.mkrf.ru/maps/show/id/4187985
4,"Здание городского торгового корпуса, где 14 де...",541510302250006,"Новосибирская область, г. Новосибирск, Красный...",Новосибирская область,Федерального значения,Памятник,"1. Объемно-планировочное решение, форма и габа...",Граница территории объекта культурного наследи...,нет,нет,NaN,1917 г.,"[82.920241, 55.028734]",Памятник истории,Постановление Совета Министров РСФСР «О дополн...,624,1974-12-04,https://okn-mk.mkrf.ru/maps/show/id/122292,https://okn-mk.mkrf.ru/maps/show/id/122297


<br>

Зададим в каком порядке они будут отображены.

In [72]:
df = df[['object',
         'registry_number',
         'region',
         'full_address',
         'construction_date',
         'significance_category',
         'object_type',
         'object_subtype',
         'unesco_affiliation',
         'particularly_valuable_object',
         'object_description',
         'object_border_description',
         'ensemble_id',
         'coordinates',
         'image_url',
         'document_name',
         'document_code',
         'document_date',
         'document_url']]

<br>

Срез по г. Ухте.

In [73]:
df[df['full_address'].str\
                     .contains('Ухта', na = False)]\
                     .sort_values(by='object')\
                     .head()

,object,registry_number,region,full_address,construction_date,significance_category,object_type,object_subtype,unesco_affiliation,particularly_valuable_object,object_description,object_border_description,ensemble_id,coordinates,image_url,document_name,document_code,document_date,document_url
147528,"Ансамбль улиц Мира, Первомайская, Кремса, архи...",112021333140005,Республика Коми,"Республика Коми, г. Ухта, ул. Мира, дома №№ 1,...",1949 - 1968 гг.,Регионального значения,Ансамбль,Памятник градостроительства и архитектуры,нет,нет,NaN,Граница территории объекта культурного наследи...,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/4034367,Приказ Управления Республики Коми по охране об...,93-ОД,2019-06-17,NaN
147483,"Ансамбль улиц Октябрьская и Первомайская, архи...",112021333120005,Республика Коми,"Республика Коми, г. Ухта, ул. Октябрьская, дом...",1950 - 1959 гг.,Регионального значения,Ансамбль,Памятник градостроительства и архитектуры,нет,нет,Предметом охраны объекта культурного наследия...,NaN,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/4028963,Приказ Управления Республики Коми по охране об...,167-ОД,2019-08-05,NaN
149208,Больница поселка Водный,112111343810005,Республика Коми,"Республика Коми, г. Ухта, пгт. Водный, ул. Гаг...",1956 г.,Регионального значения,Памятник,Памятник градостроительства и архитектуры,нет,нет,1 Объемно-пространственная композиция здания ...,Граница территории объекта культурного наследи...,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/4396147,Приказ Управления Республики коми по охране об...,20-ОД,2021-03-05,NaN
71930,Братская могила 12 красных партизан в с. Изваиле,111610537090005,Республика Коми,"Республика Коми, г. Ухта, д. Изваиль (ориентир...","ноябрь 1919 г., ноябрь 1957 г.",Регионального значения,Памятник,Памятник истории,нет,нет,"К особенностям, составляющим предмет охраны об...",NaN,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/1962138,"Постановление Совета Министров Коми АССР ""О па...",406,1959-11-30,NaN
93197,Братская могила 7 красных партизан в с. Усть-Ухте,111710819530005,Республика Коми,"Республика Коми, г. Сосногорск, с. Усть-Ухта, ...","1919 г., 1957 г.",Регионального значения,Памятник,Памятник истории,нет,нет,"К особенностям, составляющим предмет охраны об...",NaN,NaN,NaN,https://okn-mk.mkrf.ru/maps/show/id/2455551,"Постановление Совета Министров Коми АССР ""О па...",406,1959-11-30,NaN


<br>

#### Изменение типов данных

Нужно проверить типы данных и поменять их там, где нужно.

In [74]:
# типы данных
df.dtypes

object                           object
registry_number                  object
region                           object
full_address                     object
construction_date                object
significance_category            object
object_type                      object
object_subtype                   object
unesco_affiliation               object
particularly_valuable_object     object
object_description               object
object_border_description        object
ensemble_id                     float64
coordinates                      object
image_url                        object
document_name                    object
document_code                    object
document_date                    object
document_url                     object
dtype: object

Кажется, неправильный тип данных есть в колонке `registry_number`, но в некоторых её значениях присутствуют посторонние знаки, а не только цифры.\
Да и математические операции не нужны этой колонке, ведь это номер в реестре.

Неправильные типы данных есть в колонках:
* `ensemble_id`;
* `coordinates`;
* `document_date`.

<br>

**Преобразуем типы.**

Идентификатор ансамбля.

In [75]:
df['ensemble_id'] = df['ensemble_id'].astype('Int64')

<br>

Координаты предоставлены в виде списка, преобразуем в строку.

In [76]:
df['coordinates'] = df['coordinates'].astype('str')

<br>

Даты документов. Без значения **coerce** параметра **errors** не получится преобразовать тип, потому что есть аномальные значения.\
В таком случае эти аномальные значения заменятся на пропуски.

Количество пропусков до преобразования.

In [77]:
df['document_date'].isna().sum()

1580

Посмотрим уникальные значения дат.

In [78]:
udates = df['document_date'].sort_values().unique().tolist()
udates[:30]

['0198-04-06',
 '0198-11-01',
 '0201-07-05',
 '0213-10-15',
 '0998-02-11',
 '1005-08-14',
 '1007-01-28',
 '1007-11-27',
 '1019-03-13',
 '1067-08-21',
 '1071-08-27',
 '1080-04-28',
 '1191-10-25',
 '1193-04-15',
 '1196-03-12',
 '1327-08-30',
 '1597-11-14',
 '1669-01-30',
 '1690-08-30',
 '1791-09-29',
 '1852-06-18',
 '1860-08-30',
 '1873-08-20',
 '1886-07-24',
 '1893-04-21',
 '1900-11-05',
 '1935-01-01',
 '1940-10-07',
 '1946-04-25',
 '1947-08-27']

Ну вот, видны неправдоподобные даты. Неизвестно, с какого года существует такая программа у России по сохранению объектов.\
Тут видны очевидные опечатки.\
Также здесь есть такие даты, которые метод `pd.to_datetime` не может преобразовать, так как они находятся за пределами допустимого диапазона для временных меток в Pandas (с $1677$ года до $2262$ года).

In [79]:
df['document_date'] = pd.to_datetime(df['document_date'], format='%Y-%m-%d', errors='coerce')

Количество пропусков после преобразования.

In [80]:
df['document_date'].isna().sum()

1609

Добавилось $29$ новых пропусков. Совсем ничего страшного. Капля в море.

С типами данных разобрались.

<br>

### Пропущенные значения

Наконец-то можно спокойно проанализировать пропущенные значения.\
Отобразим сводную таблицу с количеством пропусков и их процентом от общего числа наблюдений в колонке.

In [81]:
null_values = pd.concat([df.isna().sum(), df.isna().mean() * 100], axis=1)
null_values.columns=['missing_values', 'percent']
null_values

,missing_values,percent
object,0,0.000000
registry_number,0,0.000000
region,0,0.000000
full_address,56096,36.309736
construction_date,1868,1.209116
significance_category,0,0.000000
object_type,0,0.000000
object_subtype,3542,2.292661
unesco_affiliation,11,0.007120
particularly_valuable_object,11,0.007120


Видим большой процент пропусков в `описании объектов`, `описании границ объектов`, `идентификаторе ансамбля`, `ссылке на документ`.

Пока что понятно точно, почему есть пропуски в `ensemble_id`, ведь далеко не каждый объект является объектом какого-то архитектурного ансамбля.

Но по остальным упомянутым колонкам есть вопросы. Тут несколько причин может быть:
1. Единый государственный реестр по какой-то причине не занёс эту информацию.
2. Работники-бюджетники не удосужились как-то описать объекты (саркастично: зачем, ведь и так сойдёт)

Одно мы знаем точно, государство испытывает трудности, раз данные единого реестра не полны.

Посмотрим, почему есть пропуски в `полном адресе`.\
Есть подозрение, что это из-за того, что общая видовая принадлежность таких объектов (`object_subtype`) — это памятник археологии. Это значит, что такие места расположены вне населённых пунктов.

In [82]:
df[df['object_subtype'] == 'Памятник археологии'].isna().sum()

object                              0
registry_number                     0
region                              0
full_address                    56096
construction_date                 256
significance_category               0
object_type                         0
object_subtype                      0
unesco_affiliation                  0
particularly_valuable_object        0
object_description              56096
object_border_description       56096
ensemble_id                     54244
coordinates                         0
image_url                           0
document_name                       3
document_code                     165
document_date                     241
document_url                    56096
dtype: int64

И действительно, это так. Да, в таком случае `полный адрес` никак не указать. Пропуски здесь органично вписываются.

Пропуски в `годе постройки` могут быть из-за того, что доподлинно неизвестно, когда было построено то или иное сооружение.

В общем, с пропусками ничего не поделать. Оставить как есть.

<br>

### Дублирующие значения

Посмотрим, есть ли клоны среди записей.

In [83]:
df.duplicated().sum()

0

Нет, полных дублей нет. Все записи оригинальны, т.е. исключаем вероятность, что какой-то объект по ошибке повторяется в датасете.

<br>

Посмотрим, попадутся ли на глаза неявные дубли.

Создадим функцию для поиска.

In [84]:
def duplicates(column):
    print(df[column].value_counts(dropna=False).head(10))
    print('')
    print(f'Всего значений: {df[column].count()}')
    print(f'Количество уникальных значений: {df[column].nunique()}')

<br>

**Посмотрим колонку `object`.**

In [85]:
duplicates('object')

Дом жилой                           4021
Жилой дом                           2113
Курган                              1381
Курганная группа                     887
Селище                               755
Городище                             713
Памятник В.И. Ленину                 668
Братская могила советских воинов     577
Флигель                              543
Главный дом                          533
Name: object, dtype: int64

Всего значений: 154493
Количество уникальных значений: 104510


Дом жилой и Жилой дом, очевидно, одно и то же. Это можно объединить.\
А вот Курган и Курганная группа — вещи немного разные, как подсказывает гугл.\
Уникальных значений много. Дополнительно выискивать дубли среди них кажется неразумным.

Заменим **Дом жилой** на **Жилой дом**.

In [86]:
df['object'] = df['object'].replace('Дом жилой', 'Жилой дом')

И проверим снова.

In [87]:
duplicates('object')

Жилой дом                           6134
Курган                              1381
Курганная группа                     887
Селище                               755
Городище                             713
Памятник В.И. Ленину                 668
Братская могила советских воинов     577
Флигель                              543
Главный дом                          533
Братская могила                      527
Name: object, dtype: int64

Всего значений: 154493
Количество уникальных значений: 104509


<br>

**Посмотрим колонку `registry_number`.**

In [88]:
duplicates('registry_number')

671510201120006    1
131710904690005    1
131710904710005    1
581710920830005    1
131710904720005    1
581710902210005    1
581710902830005    1
131710904780005    1
601740900280006    1
781720971980006    1
Name: registry_number, dtype: int64

Всего значений: 154493
Количество уникальных значений: 154493


Дублей нет. Каждое значение этой колонки уникально.

<br>

**Посмотрим колонку `region`.**

In [89]:
duplicates('region')

Ростовская область      9658
Краснодарский край      9264
Тверская область        7398
Республика Дагестан     6369
г. Москва               6228
г. Санкт-Петербург      6027
Алтайский край          4512
Псковская область       4322
Московская область      3900
Новгородская область    3889
Name: region, dtype: int64

Всего значений: 154493
Количество уникальных значений: 86


Кажется, все 86 регионов есть.

<br>

**Посмотрим колонку `full_address`.**

In [90]:
duplicates('full_address')

NaN                                                                                                                                                                                                                             56096
г. Москва, Новодевичье кладбище                                                                                                                                                                                                   286
г. Севастополь, Братское кладбище защитников Севастополя 1854-1855 гг. Северная сторона                                                                                                                                           151
г. Санкт-Петербург, Александра Невского пл., 1                                                                                                                                                                                    126
Республика Ингушетия, Джейрахский муниципальный район, с. п. Гули, с. Мусийкъонг

Многие объекты находятся в определённых местах. В будущем нужно подумать, как их все разместить на карте.

<br>

**Посмотрим колонку `construction_date`.**

In [91]:
duplicates('construction_date')

дата создания (возникновения) не определена    8114
XIX в.                                         4027
III тыс. до н.э. - XIV в. н.э.                 3096
Дата создания (возникновения) не определена    2236
1941-1945 гг.                                  2149
NaN                                            1868
1943 г.                                        1827
кон. XIX в.                                    1798
эпоха бронзы - позднее средневековье           1411
нач. XX в.                                     1346
Name: construction_date, dtype: int64

Всего значений: 152625
Количество уникальных значений: 26598


Запишем кол-во уникальных значений до обработки.

In [92]:
const_date_before = df["construction_date"].nunique()

Эта колонка — чемпион среди колонок с дублями.\
Приведём все символы, за исключением римских цифр, к нижнему регистру.

In [93]:
# преобразование 'construction_date' в тип 'str', чтобы избежать казусов
df['construction_date'] = df['construction_date'].astype('str')

# функция для приведения строки к нижнему регистру, исключая символы 'I', 'X', 'V', 'Х'
def custom_lowercase(text):
    # компиляция регулярного выражения для поиска всех символов, кроме 'I', 'X', 'V', 'Х', игнорируя регистр
    pattern = re.compile('[^IXVХ]+', re.IGNORECASE)
    # применение регулярного выражения для замены найденных символов на их нижний регистр
    # метод sub() выполняет замену; лямбда ф-я принимает x и возвращает его группу (подстроку) в нижнем регистре; text — строка, в которой идёт замена
    result = pattern.sub(lambda x: x.group(0).lower(), text)
    return result

# применение функции к колонке 'construction_date'
df['construction_date'] = df['construction_date'].apply(lambda x: custom_lowercase(x))

In [94]:
df['construction_date'].isna().sum()

0

При этом пропущенные значения стали строками. Нужно исправить это.

In [95]:
df['construction_date'] = df['construction_date'].replace('nan', np.nan)

In [96]:
df['construction_date'].isna().sum()

1868

Пропуски вернулись.

Отобразим подробное распределение значений в `construction_date`.

In [97]:
df['construction_date'].value_counts().head(50)

дата создания (возникновения) не определена    10350
XIX в.                                          4027
III тыс. до н.э. - XIV в. н.э.                  3096
1941-1945 гг.                                   2149
кон. XIX в.                                     1922
1943 г.                                         1827
нач. XX в.                                      1430
эпоха бронзы - позднее средневековье            1412
конец XIX в.                                    1408
III тыс. до н. э. -  XIV в. н. э.               1305
начало XX в.                                    1252
эпоха бронзы                                    1244
XVIII в.                                        1092
II пол. I тыс. н.э.                              995
конец XIX века                                   949
неолит                                           928
III тыс. до н.э. - XIV в. н. э.                  894
средние века                                     889
III тыс. до н.э.                              

**Устраним все неявные дубликаты, где содержится от 100 и больше значений.**

In [98]:
df['construction_date'] = df['construction_date'].replace({'III тыс. до н. э. - XIV в. н. э.':         'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н. э. -  XIV в. н. э.':        'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н.э.-XIV в. н.э.':             'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н.э. - XIV в. н. э.':          'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н.э. - XIV вв. н.э.':          'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н.э. - XIV в.н.э.':            'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н.э. – XIV в. н.э.':           'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н.э. -  XIV в. н.э.':          'III тыс. до н.э. - XIV в. н.э.',
                                                           'III тыс. до н.э. -  XIV в.н.э.':           'III тыс. до н.э. - XIV в. н.э.',
                                                           'третье тысячелетие до н.э. - XIV в. н.э.': 'III тыс. до н.э. - XIV в. н.э.',
                                                           'XVIII век':                                'XVIII в.',
                                                           'конец XVIII-начало XIX вв.':               'кон. XVIII-нач. XIX вв.',
                                                           'XVIII - XIX вв.':                          'XVIII-XIX вв.',
                                                           'XIX век':                                  'XIX в.',
                                                           'середина XVIII в.':                        'сер. XVIII в.',
                                                           'вторая половина XVIII в.':                 '2-я пол. XVIII в.',
                                                           'первая половина XIX в.':                   '1-я пол. XIX в.',
                                                           'пер. пол. XIX в.':                         '1-я пол. XIX в.',
                                                           'I пол. XIX в.':                            '1-я пол. XIX в.',
                                                           'вторая половина XIX века':                 '2-я пол. XIX в.',
                                                           '2-я половина XIX в.':                      '2-я пол. XIX в.',
                                                           'II пол. XIX в.':                           '2-я пол. XIX в.',
                                                           'вт. пол. XIX в.':                          '2-я пол. XIX в.',
                                                           'вторая половина XIX в.':                   '2-я пол. XIX в.',
                                                           'начало XIX века':                          'нач. XIX в.',
                                                           'середина XIX века':                        'сер. XIX в.',
                                                           'конец XIX века':                           'кон. XIX в.',
                                                           'к. XIX в.':                                'кон. XIX в.',
                                                           'кон. XIX-нач. XX вв.':                     'кон. XIX - нач. XX вв.',
                                                           'конец XIX - начало ХХ вв.':                'кон. XIX - нач. XX вв.',
                                                           'конец XIX-начало XX века':                 'кон. XIX - нач. XX вв.',
                                                           'нач. ХХ в.':                               'нач. XX в.',
                                                           'начало XX века':                           'нач. XX в.',
                                                           'начало ХХ века':                           'нач. XX в.',
                                                           'начало ХХ в.':                             'нач. XX в.',
                                                           'н. XX в.':                                 'нач. XX в.',
                                                           'средние века':                             'средневековье',
                                                           'эпоха бронзы':                             'бронзовый век',
                                                           '1942 - 1943 годы':                         '1942-1943 гг.',
                                                           'III - II тыс. до н.э.':                    'III-II тыс. до н.э.',
                                                           'XIV - XVI вв.':                            'XIV-XVI вв.',
                                                           'эпоха железа':                             'железный век',
                                                           'эпоха неолита':                            'неолит',
                                                           'эпоха раннего железа':                     'ранний железный век',
                                                           'эпоха средневековья':                      'средневековье',
                                                           '1941-1943 гг.':                            '1941-1945 гг.',
                                                           '1941-1944 гг.':                            '1941-1945 гг.',
                                                           '1942-1943 гг.':                            '1941-1945 гг.',
                                                           '1941 - 1943 гг.':                          '1941-1945 гг.',
                                                           '1944 год':                                 '1944 г.',
                                                           '-':                                        'дата создания (возникновения) не определена',
                                                           'XII - XVI вв.':                            'XII-XVI вв.',
                                                           'XVIII – XX вв.':                           'XVIII-XX вв.',
                                                           'эпоха палеолита':                          'палеолит',
                                                           'эпоха позднего средневековья':             'позднее средневековье',
                                                           'эпоха поздней бронзы':                     'поздний бронзовый век'})

df['construction_date'] = df['construction_date'].str.replace(' - ', '-')\
                                                 .str.replace(' – ', ' -')\
                                                 .str.replace('год', 'г.')\
                                                 .str.replace('Х', 'X')\
                                                 .str.replace('г.ы', 'гг.', regex=True)\
                                                 .str.replace('начало', 'нач.')\
                                                 .str.replace('середина', 'сер.')\
                                                 .str.replace('половина', 'пол.')\
                                                 .str.replace('2 -я', '2-я')\
                                                 .str.replace('2- я', '2-я')\
                                                 .str.replace('вторая', '2-я')\
                                                 .str.replace('2 пол.', '2-я пол', regex=True)\
                                                 .str.replace('II пол.', '2-я пол', regex=True)\
                                                 .str.replace('2-ая', '2-я')\
                                                 .str.replace('II-я пол.', '2-я пол', regex=True)\
                                                 .str.replace('1 -я', '1-я')\
                                                 .str.replace('1- я', '1-я')\
                                                 .str.replace('первая', '1-я')\
                                                 .str.replace('1 пол.', '1-я пол', regex=True)\
                                                 .str.replace('I пол.', '1-я пол', regex=True)\
                                                 .str.replace('I-я пол.', '1-я пол', regex=True)\
                                                 .str.replace('1-ая', '1-я')\
                                                 .str.replace('четверть', 'четв.')\
                                                 .str.replace('I1-я пол', '2-я пол.')

# удалим пробелы первым и последним символом в значениях
df['construction_date'] = df['construction_date'].str.strip(' ')

Код ниже помогал искать неявные дубликаты, в `head()` задавалось количество отображаемых строк, а `set_option` снимало ограничения.\
Отсортированная в алфавитном порядке таблица помогала выявить неявные дубликаты.

In [99]:
pd.set_option('display.max_rows', None)
duplicates_search = df['construction_date'].value_counts(dropna=False).reset_index()
duplicates_search = duplicates_search.sort_values(by='index')
pd.reset_option('display.max_rows')

Вновь посмотрим распределение.

In [100]:
df['construction_date'].value_counts().head(50)

дата создания (возникновения) не определена    10501
III тыс. до н.э.-XIV в. н.э.                    7865
XIX в.                                          4646
нач. XX в.                                      4408
1941-1945 гг.                                   3438
кон. XIX в.                                     3287
2-я пол. XIX в.                                 2064
1943 г.                                         1900
средневековье                                   1778
бронзовый век                                   1632
конец XIX в.                                    1464
сер. XIX в.                                     1442
эпоха бронзы-позднее средневековье              1412
XVIII в.                                        1256
неолит                                          1111
2-я пол I тыс. н.э.                             1006
ранний железный век                              995
кон. XIX-нач. XX вв.                             900
нач. XIX в.                                   

Итоговая картина такая.\
Поиск неявных дубликатов в этой колонке может продолжаться долго.\
И лучшего способа я пока не знаю.

Cколько дубликатов было удалено.

In [101]:
const_date_after = df["construction_date"].nunique()
print(f'Количество уникальных значений до обработки: {const_date_before}')
print(f'Количество уникальных значений после обработки: {const_date_after}')
print(f'Неявных дубликатов убрано: {const_date_before - const_date_after}')

Количество уникальных значений до обработки: 26598
Количество уникальных значений после обработки: 22631
Неявных дубликатов убрано: 3967


<br>

**Посмотрим колонку `significance_category`.**

In [102]:
duplicates('significance_category')

Регионального значения                78696
Федерального значения                 71746
Местного (муниципального) значения     4051
Name: significance_category, dtype: int64

Всего значений: 154493
Количество уникальных значений: 3


Дублей нет.

<br>

**Посмотрим колонку `object_type`.**

In [103]:
duplicates('object_type')

Памятник                     129762
Ансамбль                      22382
Достопримечательное место      2349
Name: object_type, dtype: int64

Всего значений: 154493
Количество уникальных значений: 3


Дублей нет.

<br>

**Посмотрим колонку `object_subtype`.**

In [104]:
duplicates('object_subtype')

Памятник градостроительства и архитектуры    56310
Памятник археологии                          56096
Памятник истории                             35002
None                                          3542
Памятник искусства                            3442
Памятник религии                               101
Name: object_subtype, dtype: int64

Всего значений: 150951
Количество уникальных значений: 5


Дублей нет.

<br>

**Посмотрим колонку `unesco_affiliation`.**

In [105]:
duplicates('unesco_affiliation')

нет    153534
да        948
NaN        11
Name: unesco_affiliation, dtype: int64

Всего значений: 154482
Количество уникальных значений: 2


Дублей нет.

<br>

**Посмотрим колонку `particularly_valuable_object`.**

In [106]:
duplicates('particularly_valuable_object')

нет    154217
да        265
NaN        11
Name: particularly_valuable_object, dtype: int64

Всего значений: 154482
Количество уникальных значений: 2


Дублей нет.

<br>

**Посмотрим колонку `object_description`.**

In [107]:
duplicates('object_description')

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Вижу 5 дублей в разных вариантах: `не утверждён`. Исправим.

In [108]:
df['object_description'] = df['object_description'].replace('не утвержден',                 'Предмет охраны не утверждён')\
                                                   .replace('Не утвержден.',                'Предмет охраны не утверждён')\
                                                   .replace('Не утвержден' ,                'Предмет охраны не утверждён')\
                                                   .replace('Предмет охраны не утвержден.', 'Предмет охраны не утверждён')\
                                                   .replace('Предмет охраны не утвержден',  'Предмет охраны не утверждён')

Устранённые дубли должны образоваться в самое частотное значение. Проверим.

In [109]:
df['object_description'].value_counts().head(1)

Предмет охраны не утверждён    4212
Name: object_description, dtype: int64

<br>

**Посмотрим колонку `object_border_description`.**

In [110]:
duplicates('object_border_description')

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        112258
Не утверждены.                                                                                        

Видим похожую картину с `не утверждёнными` границами.

In [111]:
df['object_border_description'] = df['object_border_description'].replace('Не утверждены.',                     'Границы территории не утверждены')\
                                                                 .replace('не утверждены',                      'Границы территории не утверждены')\
                                                                 .replace('Границы территории не утверждены.' , 'Границы территории не утверждены')\
                                                                 .replace('Не утверждены',                      'Границы территории не утверждены')

Устранённые дубли должны образоваться в самое частотное значение. Проверим.

In [112]:
df['object_border_description'].value_counts().head(1)

Границы территории не утверждены    2883
Name: object_border_description, dtype: int64

<br>

**Посмотрим колонку `ensemble_id`.**

In [113]:
duplicates('ensemble_id')

<NA>      132057
151344       219
164457       147
214508       121
194299       107
215413       103
169364        95
212661        93
220283        90
216087        83
Name: ensemble_id, dtype: Int64

Всего значений: 22436
Количество уникальных значений: 4953


Кажется, порядок. Если объект является частью ансамбля, то ему присуждается определённый номер.\
Так, видим, что есть $219$ объектов одного ансамбля.

<br>

**Посмотрим колонку `coordinates`.**

In [114]:
duplicates('coordinates')

nan                                     114543
[45.450136, 42.707692]                      69
[49.1068469365817, 55.7999436090671]        47
[34.9578965608516, 44.8417174338509]        45
[37.788144, 55.693409]                      39
[30.326244, 59.864272]                      33
[43.2463865344229, 55.040424834587]         31
[47.970126, 46.4657]                        28
[43.3490221878354, 56.1642316596241]        27
[34.250714, 61.816268]                      23
Name: coordinates, dtype: int64

Всего значений: 154493
Количество уникальных значений: 36406


Множество объектов имеют одни и те же координаты. Очень много пропусков. Даже если есть дубли, исправить их не представляется возможным.

<br>

**Посмотрим колонку `image_url`.**

In [115]:
duplicates('image_url')

                                               60542
https://okn-mk.mkrf.ru/maps/show/id/1836648        6
https://okn-mk.mkrf.ru/maps/show/id/1780885        3
https://okn-mk.mkrf.ru/maps/show/id/1793099        3
https://okn-mk.mkrf.ru/maps/show/id/4546450        2
https://okn-mk.mkrf.ru/maps/show/id/3818215        2
https://okn-mk.mkrf.ru/maps/show/id/4401774        2
https://okn-mk.mkrf.ru/maps/show/id/979207         2
https://okn-mk.mkrf.ru/maps/show/id/4727699        2
https://okn-mk.mkrf.ru/maps/show/id/409880         2
Name: image_url, dtype: int64

Всего значений: 154493
Количество уникальных значений: 93935


Пустое значение повторяется $60542$ раза. Это пропуск, у этого объекта нет изображения. Нужно переименовать это пустое значение.\
Даже если дубли есть, исправить их не представляется возможным.

In [116]:
df['image_url'] = df['image_url'].replace('', 'Нет изображения')

<br>

**Посмотрим колонку `document_name`.**

In [117]:
duplicates('document_name')

Закон Краснодарского края «О перечне объектов культурного наследия (памятников истории и культуры), расположенных на территории Краснодарского края»                                                                 8182
Постановление                                                                                                                                                                                                        3503
Постановление Главы Администрации Ростовской области «О принятии на государственную охрану памятников истории и культуры области и мерах по их охране»                                                               3485
Постановление Законодательного собрания Тверской области                                                                                                                                                             3415
Постановление Псковского областного Собрания депутатов «Об утверждении государственного списка недвижимых памятников истории и к

Кажется, тут порядок. Видим, что один и тот же документ назначает очень много объектов.

<br>

**Посмотрим колонку `document_url`.**

In [118]:
duplicates('document_url')

NaN                                            58475
None                                           56096
https://okn-mk.mkrf.ru/maps/show/id/2137890      283
https://okn-mk.mkrf.ru/maps/show/id/2113631      269
https://okn-mk.mkrf.ru/maps/show/id/748077       164
https://okn-mk.mkrf.ru/maps/show/id/367586       112
https://okn-mk.mkrf.ru/maps/show/id/3423155       97
https://okn-mk.mkrf.ru/maps/show/id/598474        73
https://okn-mk.mkrf.ru/maps/show/id/903822        53
https://okn-mk.mkrf.ru/maps/show/id/459615        51
Name: document_url, dtype: int64

Всего значений: 39922
Количество уникальных значений: 37493


Тут тоже не наблюдаются дубли.

<br>

## Рефлексия

Это очень хорошо, что занялись предобработкой всех данных. Неплохая тренировка.

Но напрашивается вопрос, все ли колонки по  большому счёту нужны?

К тому же, некоторые из них очень громоздкие.

Давайте посмотрим, сколько памяти использует датафрейм и его колонки.

In [119]:
# общий размер памяти в мегабайтах, занимаемый df
mem_us_after = df.memory_usage(deep=True).sum() / (1024 * 1024)

# размер памяти в мегабайтах, занимаемый каждой колонкой
col_mem = df.memory_usage(deep=True) / (1024 * 1024)

# размер памяти в процентах, занимаемый каждой колонкой
col_mem_perc = col_mem / mem_us_after * 100

col_mem_perc

Index                            0.000020
object                           5.242539
registry_number                  1.703939
region                           3.660096
full_address                     3.557487
construction_date                2.750086
significance_category            4.007402
object_type                      2.745849
object_subtype                   3.134223
unesco_affiliation               2.082337
particularly_valuable_object     2.082546
object_description              39.399015
object_border_description       13.195174
ensemble_id                      0.213006
coordinates                      1.592116
image_url                        2.475933
document_name                    9.277044
document_code                    1.590577
document_date                    0.189338
document_url                     1.101274
dtype: float64

Колонки `object_description` и `object_border_description` занимают больше половины всей таблицы.

Информация о документах в колонках `document_name`, `document_code`, `document_date`, `document_url` поможет подробно ознакомиться с объектом.

Ссылки на изображения объектов `image_url` и id ансамблей `ensemble_id` также выполняют схожую функцию: дополнительная информация об объектах.

Но отпустим все эти колонки. Если заинтересует какой-то объект, о нём можно узнать подробнее в открытых данных минкульта РФ.

In [120]:
df.drop(columns=['object_description',
                 'object_border_description',
                 'document_name',
                 'document_code',
                 'document_date',
                 'document_url',
                 'image_url',
                 'ensemble_id'], inplace=True)

<br>

Сравним объём датафрейма до обработки и после обработки.

In [121]:
# перезапишем переменную после удаления колонок
mem_us_after = df.memory_usage(deep=True).sum() / (1024 * 1024)

print(f'{round(mem_us_before)} Мб — такой объём памяти занимала таблица до обработки.')
print(f'{round(mem_us_after)} Мб — такой объём памяти занимает таблица сейчас, после обработки.')
print(f'Таким образом, теперь таблица весит на {int((1 - round(mem_us_after / mem_us_before, 2)) * 100)} % меньше.')

830 Мб — такой объём памяти занимала таблица до обработки.
203 Мб — такой объём памяти занимает таблица сейчас, после обработки.
Таким образом, теперь таблица весит на 76 % меньше.


In [122]:
print(f'Размер таблицы до предобработки: {raw_shape[0]} строк и {raw_shape[1]} колонок.')
print(f'Размер таблицы после предобработки: {df.shape[0]} строк и {df.shape[1]} колонок.')

Размер таблицы до предобработки: 154493 строк и 53 колонок.
Размер таблицы после предобработки: 154493 строк и 11 колонок.


Не потеряли ни одной строки, но удалили 42 колонки.

<br>

## Резюме по предобработке данных

**Что было сделано:**
* снесли колонки, почти полностью состоящих из пропущенных значений;
* преобразовали JSON-данные, отчего появились новые колонки;
* после преобразования снова очистили датафрейм от ненужных колонок, где было очень много пропусков;
* разобрались, какую информацию преподносят колонки;
* углубившись в суть, дополнительно ещё убрали мусорные колонки;
* переименовали колонки, упорядочили их;
* в каких-то колонках изменили тип данных;
* проанализировали пропущенные значения;
* проанализированы неявные дубликаты и устранили их, где возможно;
* сравнили исходный и обработанный датафреймы.

Таким образом, данные готовы для анализа.

Экспортируем обработанный датафрейм для дальнейшего анализа.

In [123]:
# экспорт
df.to_csv('/content/drive/MyDrive/Colab Datasets/clean_cultural.csv', index=False)

Увидимся во второй части!